In [1]:
import pandas as pd
import pandasql as sql
data = pd.read_csv('../data/owid-co2-data.csv')
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [34]:
# filtrer etter rader som har iso kode og et land
country_query = """
SELECT DISTINCT iso_code, country, population, co2_growth_abs, year
FROM data
WHERE iso_code IS NOT NULL AND iso_code != "" AND LENGTH(iso_code) = 3
GROUP BY iso_code, year
"""
country_result = sql.sqldf(country_query, locals())
# country_result['country'].unique()

In [ ]:
# To find the iso_codes. Used to filter out European countries, and then into north- or south.
iso_query = """
SELECT DISTINCT iso_code, country
FROM data
WHERE iso_code IS NOT NULL AND iso_code != "" AND LENGTH(iso_code) = 3
"""
country_result = sql.sqldf(iso_query, locals())
iso_codes = country_result['iso_code'].unique()
# print(iso_codes)

In [7]:
# Create a mapping of north & south.
northern_europe = ['NOR', 'SWE', 'FIN', 'DNK', 'ISL', 'GBR', 'IRL', 'DEU', 'NLD', 'POL', 'EST', 'LVA', 'LTU', 'RUS', 'BLR', 'CZE', 'SVK']
southern_europe = ['ITA', 'ESP', 'PRT', 'GRC', 'MLT', 'CYP', 'HRV', 'SVN', 'BGR', 'ROU', 'ALB', 'BIH', 'MNE', 'MKD']

# Add a 'hemisphere' column based on the north & south mapping.
country_result['north_or_south'] = country_result['iso_code'].apply(lambda x: 'Northern' if x in northern_europe else ('Southern' if x in southern_europe else None))

# Filter out to only grab countries with a hemisphere column.
europe_result = country_result[country_result['north_or_south'].notnull()]


In [14]:
# print(europe_result)

In [23]:
# Query the "ghg_per_capita" - Total greenhouse gas emissions per capita including land-use change and forestry - Emissions are measured in tonnes of carbon dioxide-equivalents per person.
ghg_query = """
SELECT DISTINCT iso_code, country, year, population, ghg_per_capita
FROM data
WHERE iso_code IS NOT NULL AND iso_code != "" AND LENGTH(iso_code) = 3
GROUP BY iso_code, year
"""
ghg_result = sql.sqldf(ghg_query, locals())

# Add the 'hemisphere' column based on your mapping
ghg_result['north_or_south'] = ghg_result['iso_code'].apply(lambda x: 'Northern' if x in northern_europe else ('Southern' if x in southern_europe else None))

# Filter for European countries that have a hemisphere column.
ghg_result = ghg_result[ghg_result['north_or_south'].notnull()]



In [ ]:
# print(country_result)# Filter rows where 'ghg_per_capita' is not null and sort by year
first_ghg_data = ghg_result[ghg_result['ghg_per_capita'].notnull()].sort_values(by='year')

# Display the first rows to see the earliest entries
""" first_entries = first_ghg_data.head(10)
print(first_entries) """


      iso_code          country  year  population  total_ghg north_or_south
830        ALB          Albania  1990   3295073.0      11.37       Southern
4549       BLR          Belarus  1990  10428527.0     128.92       Northern
21294      LVA           Latvia  1990   2689398.0      15.09       Northern
32941      SVK         Slovakia  1990   5261310.0      57.28       Northern
9409       CYP           Cyprus  1990    788501.0       5.37       Southern
17472      ITA            Italy  1990  56756560.0     448.74       Southern
22955      MKD  North Macedonia  1990   2044181.0      13.90       Southern
20944      LTU        Lithuania  1990   3785851.0      51.82       Northern
16607      IRL          Ireland  1990   3485370.0      64.56       Northern
23647      MNE       Montenegro  1990    621450.0       4.15       Southern


# To look at ghg_per_capita
"Total greenhouse gas emissions per capita, including land-use change and forestry, measured in tonnes of CO2-equivalents per person."

- We can compare greenhouse gas emissions on a per-person basis, across different countries in Europe. Where we can see how the emissions scale with population. We can look at whether certain countries emit more GHG per person. 

In [33]:
# Keeping this initial graph until we figure out if we want two separate files, or the way I have done it below is ok.
""" app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("GHG Emissions per Capita by Country: Northern vs Southern Europe"),
    dcc.Graph(id='ghg-trends-graph')
])

# Callback to plot all countries as individual lines, colored by north/south location.
@app.callback(
    Output('ghg-trends-graph', 'figure'),
    # Dash wouldn't work without initial dummy data.
    Input('ghg-trends-graph', 'id')
)
def update_ghg_graph(dummy_input):
    # Create a plot line for each country, with color being red or blue depending on location.
    fig1 = px.line(
        ghg_result,
        x='year',
        y='ghg_per_capita',
        color='north_or_south',  # Ensure this matches the column in your DataFrame
        line_group='country',
        title="GHG Emissions per Capita by Country: Northern vs Southern Europe (1990 Onwards)",
        labels={'year': 'Year', 'ghg_per_capita': 'GHG per Capita (tons)'},
        color_discrete_map={'Northern': 'blue', 'Southern': 'red'}
    )

    # Customize layout to better separate lines by country
    fig1.update_traces(mode='lines', line=dict(width=1))

    # Limit the x-axis year range to start from 1990, to not show a long empty graph.
    fig1.update_xaxes(range=[1990, ghg_result['year'].max()])

    return fig1

# Run the app locally
if __name__ == '__main__':
    app.run_server(debug=True) """


' app = Dash(__name__)\n\n# Layout\napp.layout = html.Div([\n    html.H1("GHG Emissions per Capita by Country: Northern vs Southern Europe"),\n    dcc.Graph(id=\'ghg-trends-graph\')\n])\n\n# Callback to plot all countries as individual lines, colored by north/south location.\n@app.callback(\n    Output(\'ghg-trends-graph\', \'figure\'),\n    # Dash wouldn\'t work without initial dummy data.\n    Input(\'ghg-trends-graph\', \'id\')\n)\ndef update_ghg_graph(dummy_input):\n    # Create a plot line for each country, with color being red or blue depending on location.\n    fig1 = px.line(\n        ghg_result,\n        x=\'year\',\n        y=\'ghg_per_capita\',\n        color=\'north_or_south\',  # Ensure this matches the column in your DataFrame\n        line_group=\'country\',\n        title="GHG Emissions per Capita by Country: Northern vs Southern Europe (1990 Onwards)",\n        labels={\'year\': \'Year\', \'ghg_per_capita\': \'GHG per Capita (tons)\'},\n        color_discrete_map={\'No

# total_ghg from 1990 (first recorded values)
"total greenhouse gas emissions, including land-use change and forestry, measured in million tonnes of CO2-equivalents."
- If we want to look at the total emissions produced by a country as a whole (or region). We can find the countries who produce the absolute most ghg.

In [ ]:
# Query the "total_ghg"
total_ghg_query = """
SELECT DISTINCT iso_code, country, year, population, total_ghg
FROM data
WHERE iso_code IS NOT NULL AND iso_code != "" AND LENGTH(iso_code) = 3
GROUP BY iso_code, year
"""
total_ghg_result = sql.sqldf(total_ghg_query, locals())

# Add the 'north_or_south' column based on your mapping
total_ghg_result['north_or_south'] = total_ghg_result['iso_code'].apply(lambda x: 'Northern' if x in northern_europe else ('Southern' if x in southern_europe else None))

# Filter for European countries that have a "north_or_south" column.
total_ghg_result = total_ghg_result[total_ghg_result['north_or_south'].notnull()]


# Graphs for total_ghg and ghg_per_capita inside the app
In order for them to not override each other, I added both "apps" into one layout.

In [ ]:
app = Dash(__name__)

# Layout with two graphs, as I had problems with the two separate layouts overriding each other.
app.layout = html.Div([
    html.H1("GHG Emissions Analysis for Northern vs Southern Europe"),

    # ghg_per_capita
    html.H2("GHG Emissions per Capita by Country"),
    dcc.Graph(id='ghg-trends-graph'),

    # total_ghg emissions.
    html.H2("Total GHG Emissions by Country"),
    dcc.Graph(id='total-ghg-trends-graph')
])

# Callback to plot GHG Emissions per Capita for all countries
@app.callback(
    Output('ghg-trends-graph', 'figure'),
    # Dash wouldn't work without initial dummy data.
    Input('ghg-trends-graph', 'id')
)
def update_ghg_graph(dummy_input):
    # Create a line plot with each country represented as a line, with color being red or blue depending on location.
    fig1 = px.line(
        ghg_result,
        x='year',
        y='ghg_per_capita',
        color='north_or_south',  # Ensure this matches the column in your DataFrame
        line_group='country',
        title="GHG Emissions per Capita by Country: Northern vs Southern Europe (1990 Onwards)",
        labels={'year': 'Year', 'ghg_per_capita': 'GHG per Capita (tons)'},
        color_discrete_map={'Northern': 'blue', 'Southern': 'red'}
    )

    # Customize line width and opacity for better readability
    fig1.update_traces(mode='lines', line=dict(width=1))

    # Limit the x-axis year range to start from 1990, to not show a long empty graph.
    fig1.update_xaxes(range=[1990, ghg_result['year'].max()])

    return fig1

# Callback to plot Total GHG Emissions for all countries
@app.callback(
    Output('total-ghg-trends-graph', 'figure'),
    # Dash wouldn't work without initial dummy data.
    Input('total-ghg-trends-graph', 'id')
)
def update_total_ghg_graph(dummy_input):
    # Create a line plot with each country represented as a line, with color being red or blue depending on location
    fig2 = px.line(
        total_ghg_result,
        x='year',
        y='total_ghg',
        color='north_or_south',
        line_group='country',
        title="Total GHG Emissions by Country: Northern vs Southern Europe (1990 Onwards)",
        labels={'year': 'Year', 'total_ghg': 'Total GHG Emissions (million tonnes)'},
        color_discrete_map={'Northern': 'blue', 'Southern': 'red'}
    )

    # Customize line width and opacity for better readability
    fig2.update_traces(mode='lines', line=dict(width=1))

    # Limit the x-axis year range to start from 1990, to not show a long empty graph.
    fig2.update_xaxes(range=[1990, total_ghg_result['year'].max()])

    return fig2

# Run the app locally for now, will change to web later after our development phase.
if __name__ == '__main__':
    app.run_server(debug=True)



In [ ]:
# filtrer etter rader som har iso kode og et land
""" 
country_result = sql.sqldf(country_query, locals())

# Initialize the Dash app
app = Dash(__name__)

# Define country options for the dropdown
country_options = [{'label': country, 'value': country} for country in country_result['country'].unique()]

# Define the layout
app.layout = html.Div([
    html.H1("CO₂ Growth Trend Over Years by Country"),
    dcc.Dropdown(
        id='country-dropdown',
        options=country_options,
        value=country_options[0]['value'],  # Default to the first country in the list
        clearable=False
    ),
    dcc.Graph(id='co2-trend-graph')
])

# Define the callback to update the graph
@app.callback(
    Output('co2-trend-graph', 'figure'),
    Input('country-dropdown', 'value')
)
def update_graph(selected_country):
    # Filter data for the selected country
    filtered_df = country_result[country_result['country'] == selected_country]

    # Create a line plot for CO₂ growth over the years
    fig = px.line(
        filtered_df,
        x='year',
        y='co2_growth_abs',
        title=f"CO₂ Growth Trend for {selected_country}",
        labels={'year': 'Year', 'co2_growth_abs': 'CO₂ Growth Absolute'}
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True) """

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[4], line 33, in update_graph(selected_country='Aruba')
     30 filtered_df = country_result[country_result['country'] == selected_country]
     32 # Create a line plot for CO₂ growth over the years
---> 33 fig = px.line(
        px = <module 'plotly.express' from 'c:\\Python312\\Lib\\site-packages\\plotly\\express\\__init__.py'>
        filtered_df =    iso_code country hemisphere
10      ABW   Aruba       None
        selected_country = 'Aruba'
     34     filtered_df,
     35     x='year',
     36     y='co2_growth_abs',
     37     title=f"CO₂ Growth Trend for {selected_country}",
     38     labels={'year': 'Year', 'co2_growth_abs': 'CO₂ Growth Absolute'}
     39 )
     40 return fig

File c:\Python312\Lib\site-packages\plotly\express\_chart_types.py:264, in line(
    data_frame=   iso_code country hemisphere
10      ABW   A